# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

The objective of this capstone project is to analyse and select the best locations in Moscow city to open a new coffee shop. Using data science methodology and machine learning techniques like clustering, this project aims to provide solutions to answer the business question: In the city of Moscow, Russia, if a businessman is looking to open a new coffee shop, where would you recommend that they open it?

**Target Audience of this project**

This project is particularly useful to property developers and investors looking to open or invest in new coffee shops in Moscow city. This project is timely as the city is currently suffering from oversupply of coffee shops. But many of them are closing as the location is not optimal. Data from the Morgan & Mallet International Russia released last year showed that almost 20% of new opened coffee shops in 2018, were closed in the same year.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* List of neighborhoods in Moscow. This defines the scope of this project which is confined to the city of Moscow, the capital city of Russia
* Latitude and longitude coordinates of those neighbourhoods. This is required in order to plot the map and also to get the venue data
* Venue data, particularly data related to shopping malls. We will use this data to perform clustering on the neighbourhoods.

**Sources of data and methods to extract them**

This Wikipedia page (https://commons.wikimedia.org/wiki/Category:Municipal_divisions_of_Moscow) contains a list of neighbourhoods in Moscow, with a total of 134 neighbourhoods. We will use web scraping techniques to extract the data from the Wikipedia page, with the help of Python requests and beautifulsoup packages. Then we will get the geographical coordinates of the neighbourhoods using Python Geocoder package which will give us the latitude and longitude coordinates of the neighbourhoods. After that, we will use Foursquare API to get the venue data for those neighbourhoods. Foursquare has one of the largest database of 105+ million places and is used by over 125,000 developers. Foursquare API will provide many categories of the venue data, we are particularly interested in the Coffee shop category in order to help us to solve the business problem put forward. This is a project that will make use of many data science skills, from web scraping (Wikipedia), working with API (Foursquare), data cleaning, data wrangling, to machine learning (K-means clustering) and map visualization (Folium). In the next section, we will present the Methodology section where we will discuss the steps taken in this project, the data analysis that we did and the machine learning technique that was used.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import os
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
df = pd.read_excel('moscow.xlsx')
print(len(df))
df.head()

134


,Neighborhood
0,Coats of arms of municipal divisions of Moscow‎
1,Flags of municipal divisions of Moscow
2,Streets in Moscow by municipal division
3,Transport in Moscow by municipal division‎
4,Churches in Moscow by municipal division‎


In [3]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Moscow, Russia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [4]:
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

In [5]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [6]:
# merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [7]:
# check the neighborhoods and the coordinates
print(df.shape)
df

(134, 3)


,Neighborhood,Latitude,Longitude
0,Coats of arms of municipal divisions of Moscow‎,55.756960,37.615020
1,Flags of municipal divisions of Moscow,55.756960,37.615020
2,Streets in Moscow by municipal division,55.756960,37.615020
3,Transport in Moscow by municipal division‎,55.756960,37.615020
4,Churches in Moscow by municipal division‎,55.756960,37.615020
5,Municipal divisions of Moscow signs‎,55.756960,37.615020
6,Aeroport,55.788050,37.570390
7,Akademichesky‎,55.843760,37.551630
8,Alekseevsky,55.811700,37.643410
9,Altufievsky,55.879750,37.578120


In [8]:
# save the DataFrame as CSV file
df.to_csv("df_moscow.csv", index=False)

## Methodology <a name="methodology"></a>

Firstly, we need to get the list of neighbourhoods in Moscow city. Fortunately, the list is available in the Wikipedia page (https://commons.wikimedia.org/wiki/Category:Municipal_divisions_of_Moscow). We will do web scraping using Python requests and beautifulsoup packages to extract the list of neighbourhoods data. However, this is just a list of names. We need to get the geographical coordinates in the form of latitude and longitude in order to be able to use Foursquare API. To do so,
we will use the wonderful Geocoder package that will allow us to convert address into geographical coordinates in the form of latitude and longitude. After gathering the data, we will populate the data into a pandas DataFrame and then visualize the neighbourhoods in a map using Folium package. This allows us to perform a sanity check to make sure that the geographical coordinates data returned by Geocoder are correctly plotted in Moscow city. 

Next, we will use Foursquare API to get the top 100 venues that are within a radius of 2000 meters. We need to register a Foursquare Developer Account in order to obtain the Foursquare ID and Foursquare secret key. We then make API calls to Foursquare passing in the geographical coordinates of the neighbourhoods in a Python loop. Foursquare will return the venue data in JSON format and we will extract the venue name, venue category, venue latitude and longitude. With the data, we can check how many venues were returned for each neighbourhood and examine how many unique categories can be curated from all the returned venues. Then, we will analyse each neighbourhood by grouping the rows by neighbourhood and taking the mean of the frequency of occurrence of each venue category. By doing so, we are also preparing the data for use in clustering. Since we are analysing the “Coffee shop” data, we will filter the “Coffee shop” as venue category for the neighbourhoods.

Lastly, we will perform clustering on the data by using k-means clustering. K-means clustering algorithm identifies k number of centroids, and then allocates every data point to the nearest cluster, while keeping the centroids as small as possible. It is one of the simplest and popular unsupervised machine learning algorithms and is particularly suited to solve the problem for this project. We will cluster the neighbourhoods into 3 clusters based on their frequency of occurrence for “Coffee shop”. The results will allow us to identify which neighbourhoods have higher concentration of coffee shops while which neighbourhoods have fewer number of coffee shops. Based on the occurrence of coffee shops in different neighbourhoods, it will help us to answer the question as to which neighbourhoods are most suitable to open new coffee shops.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data

# Create a map of Moscow with neighborhoods superimposed on top

In [9]:
# get the coordinates of Moscow
address = 'Moscow, Russia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Moscow, Russia is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Moscow, Russia is 55.4792046, 37.3273304.


In [10]:
# create map of Toronto using latitude and longitude values
map_moscow = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_moscow)  
    
map_moscow

In [11]:
# save the map as HTML file
map_moscow.save('map_moscow.html')

# Use the Foursquare API to explore the neighborhoods

In [12]:
# define Foursquare Credentials and Version
CLIENT_ID = 'IO14Z1AQ50RN324QLSOYHPAOD3NWLYCQIW2XPJPZ44FLVXJD' # your Foursquare ID
CLIENT_SECRET = '2DYM0TKBNQCHCGF4IBM4V5TW4HO1TVTJQXQIGFELJLU2WFB0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IO14Z1AQ50RN324QLSOYHPAOD3NWLYCQIW2XPJPZ44FLVXJD
CLIENT_SECRET:2DYM0TKBNQCHCGF4IBM4V5TW4HO1TVTJQXQIGFELJLU2WFB0


Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [13]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [14]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(10879, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Coats of arms of municipal divisions of Moscow‎,55.75696,37.61502,Four Seasons,55.757005,37.616461,Hotel
1,Coats of arms of municipal divisions of Moscow‎,55.75696,37.61502,"The Ritz-Carlton, Moscow",55.757211,37.612875,Hotel
2,Coats of arms of municipal divisions of Moscow‎,55.75696,37.61502,Natura Siberica,55.758644,37.612624,Cosmetics Shop
3,Coats of arms of municipal divisions of Moscow‎,55.75696,37.61502,Manezhnaya Square (Манежная площадь),55.755697,37.615066,Plaza
4,Coats of arms of municipal divisions of Moscow‎,55.75696,37.61502,Teatralnaya Square (Театральная площадь),55.758559,37.619369,Plaza


Let's check how many venues were returned for each neighorhood

In [15]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Brateyevo‎,75,75,75,75,75,75
Butyrsky‎,100,100,100,100,100,100
Chertanovo Center‎,100,100,100,100,100,100
Chertanovo North‎,100,100,100,100,100,100
Chertanovo South‎,100,100,100,100,100,100
Cheryomushki‎,100,100,100,100,100,100
Danilovsky‎,100,100,100,100,100,100
Dmitrovsky‎,74,74,74,74,74,74
Donskoy‎,100,100,100,100,100,100


Let's find out how many unique categories can be curated from all the returned venues

In [16]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 377 uniques categories.


In [17]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Hotel', 'Cosmetics Shop', 'Plaza', 'Theater', 'Road', 'Boutique',
       'Opera House', 'Park', 'Pedestrian Plaza', 'Government Building',
       'Shopping Mall', 'Hostel', 'Event Space', 'Concert Hall',
       'Clothing Store', 'Church', 'Hookah Bar', 'Restaurant',
       'Bookstore', 'Shawarma Place', 'Salon / Barbershop', 'Café',
       'Yoga Studio', 'Russian Restaurant', 'Gymnastics Gym',
       'Coffee Shop', 'Jewelry Store', 'Design Studio', 'Art Museum',
       'Candy Store', 'Museum', 'Art Gallery', 'Historic Site',
       'Ice Cream Shop', 'Comic Shop', 'Toy / Game Store', 'Burger Joint',
       'Pizza Place', 'Arcade', 'Bridal Shop', 'Cocktail Bar',
       'Steakhouse', 'Garden', 'Adult Boutique', 'Scenic Lookout',
       'Beer Bar', 'Seafood Restaurant', "Men's Store",
       'Vietnamese Restaurant', 'Tea Room'], dtype=object)

In [18]:
venues_df[venues_df['VenueCategory']=='Coffee Shop'].count()

Neighborhood      431
Latitude          431
Longitude         431
VenueName         431
VenueLatitude     431
VenueLongitude    431
VenueCategory     431
dtype: int64

In [19]:
# check if the results contain "Coffee shop"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

Analyze Each Neighborhood

In [20]:
# one hot encoding
moscow_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
moscow_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [moscow_onehot.columns[-1]] + list(moscow_onehot.columns[:-1])
moscow_onehot = moscow_onehot[fixed_columns]

print(moscow_onehot.shape)
moscow_onehot.head()

(10879, 378)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Bath House,Bathing Area,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Blini House,Board Shop,Boat Rental,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Canal,Canal Lock,Candy Store,Car Wash,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Gym,College Track,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Cultural Center,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dairy Store,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dive Spot,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kurdish Restaurant,Lake,Latin American Restaurant,Laundry Service,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monastery,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,National Park,Nature Preserve,Neighborhood,Nightclub,Noodle House,Notary,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Gym,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Paintball Field,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Perfume Shop,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Public Art,Racecourse,Racetrack,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Roof Deck,Rugby Pitch,Rugby Stadium,Russian Restaurant,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [21]:
moscow_grouped = moscow_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(moscow_grouped.shape)
moscow_grouped

(132, 378)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Bath House,Bathing Area,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Betting Shop,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Blini House,Board Shop,Boat Rental,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Canal,Canal Lock,Candy Store,Car Wash,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,College Gym,College Track,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Cultural Center,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dairy Store,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dive Spot,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,Hill,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kurdish Restaurant,Lake,Latin American Restaurant,Laundry Service,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monastery,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,National Park,Nature Preserve,Neighborhood,Nightclub,Noodle House,Notary,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Gym,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Paintball Field,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Pelmeni House,Performing Arts Venue,Perfume Shop,Pet Service,Pet Store,Pharmacy,Photography Lab,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Public Art,Racecourse,Racetrack,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Roof Deck,Rugby Pitch,Rugby Stadium,Russian Restaurant,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,

In [22]:
len(moscow_grouped[moscow_grouped["Coffee Shop"] > 0])

105

Create a new DataFrame for Coffee shops data only

In [23]:
moscow_coffee = moscow_grouped[["Neighborhoods","Coffee Shop"]]

In [24]:
moscow_coffee.head()

,Neighborhoods,Coffee Shop
0,Brateyevo‎,0.013333
1,Butyrsky‎,0.070000
2,Chertanovo Center‎,0.030000
3,Chertanovo North‎,0.030000
4,Chertanovo South‎,0.030000


Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Moscow into 3 clusters.

In [25]:
# set number of clusters
kclusters = 3

moscow_clustering = moscow_coffee.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(moscow_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 1, 1, 1, 0, 0, 1, 1, 0])

In [26]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
moscow_merged = moscow_coffee.copy()

# add clustering labels
moscow_merged["Cluster Labels"] = kmeans.labels_

In [27]:
moscow_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
moscow_merged.head()

,Neighborhood,Coffee Shop,Cluster Labels
0,Brateyevo‎,0.013333,1
1,Butyrsky‎,0.070000,0
2,Chertanovo Center‎,0.030000,1
3,Chertanovo North‎,0.030000,1
4,Chertanovo South‎,0.030000,1


In [28]:
# merge moscow_grouped with moscow data to add latitude/longitude for each neighborhood
moscow_merged = moscow_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(moscow_merged.shape)
moscow_merged.head()

(132, 5)


,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Brateyevo‎,0.013333,1,55.637550,37.764380
1,Butyrsky‎,0.070000,0,55.807368,37.578417
2,Chertanovo Center‎,0.030000,1,55.630160,37.604520
3,Chertanovo North‎,0.030000,1,55.630160,37.604520
4,Chertanovo South‎,0.030000,1,55.630160,37.604520


In [29]:
# sort the results by Cluster Labels
print(moscow_merged.shape)
moscow_merged.sort_values(["Cluster Labels"], inplace=True)
moscow_merged

(132, 5)


,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
65,Orekhovo-Borisovo South‎,0.040000,0,55.612520,37.726390
93,Tagansky‎,0.070000,0,55.733330,37.666670
110,Aeroport,0.070000,0,55.788050,37.570390
108,Yasenevo‎,0.050000,0,55.606860,37.519910
64,Orekhovo-Borisovo North‎,0.040000,0,55.612520,37.726390
31,Kuntsevo‎,0.040000,0,55.742160,37.402250
62,Obruchevsky‎,0.040000,0,55.668760,37.518980
86,Solntsevo‎,0.044444,0,55.637110,37.381150
35,Levoberezhny‎,0.040000,0,55.881460,37.445035
74,Ramenki‎,0.060000,0,55.700000,37.500000


Finally, let's visualize the resulting clusters

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(moscow_merged['Latitude'], moscow_merged['Longitude'], moscow_merged['Neighborhood'], moscow_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

Examine Clusters

Cluster 0

In [32]:
moscow_merged.loc[moscow_merged['Cluster Labels'] == 0]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
65,Orekhovo-Borisovo South‎,0.040000,0,55.612520,37.726390
93,Tagansky‎,0.070000,0,55.733330,37.666670
110,Aeroport,0.070000,0,55.788050,37.570390
108,Yasenevo‎,0.050000,0,55.606860,37.519910
64,Orekhovo-Borisovo North‎,0.040000,0,55.612520,37.726390
31,Kuntsevo‎,0.040000,0,55.742160,37.402250
62,Obruchevsky‎,0.040000,0,55.668760,37.518980
86,Solntsevo‎,0.044444,0,55.637110,37.381150
35,Levoberezhny‎,0.040000,0,55.881460,37.445035
74,Ramenki‎,0.060000,0,55.700000,37.500000


Cluster 1

In [33]:
moscow_merged.loc[moscow_merged['Cluster Labels'] == 1]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
92,Sviblovo‎,0.030000,1,55.850000,37.633330
88,South Medvedkovo‎,0.010000,1,55.872020,37.654050
89,South Tushino‎,0.030000,1,55.853560,37.431050
81,Shchukino‎,0.010000,1,55.800000,37.450000
87,South Butovo‎,0.000000,1,55.545200,37.589000
85,Sokolniki‎,0.013514,1,55.802020,37.671590
78,Savyolki District‎,0.015152,1,55.990890,37.216330
83,Sokol ‎,0.000000,1,55.509450,37.347980
82,Silino District‎,0.000000,1,55.434110,38.714520
90,Staroye Kryukovo District‎,0.024096,1,55.979760,37.190060


Cluster 2

In [34]:
moscow_merged.loc[moscow_merged['Cluster Labels'] == 2]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
129,Zamoskvorechye‎,0.12,2,55.733330,37.633330
77,Savelovsky‎,0.13,2,55.793868,37.595486
22,Khoroshyovsky‎,0.09,2,55.778481,37.528094
24,Kitai-gorod‎,0.10,2,55.751340,37.641360
28,Krasnoselsky‎,0.09,2,55.776970,37.671830
109,Yuzhnoportovy‎,0.09,2,55.701320,37.682980
37,Lomonosovsky‎,0.08,2,55.699347,37.526222
80,Severny‎,0.10,2,55.935830,37.548890
116,Basmanny,0.10,2,55.770825,37.658813
34,Lefortovo‎,0.08,2,55.766670,37.700000


Observations:

Most of the coffee shops are concentrated in the central area of Moscow city, with the highest number in cluster 2 and moderate number in cluster 0. Cluster 1 has very low number to totally no coffee shops in the neighborhoods. This represents a great opportunity and high potential areas to open new coffee shops as there is very little to no competition from existing coffee shops. Meanwhile, coffee shops in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of coffee shops. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still has very few coffee shops. Therefore, this project recommends property developers to capitalize on these findings to open new coffee shops in neighborhoods in cluster 1 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new coffee shops in neighborhoods in cluster 0 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of coffee shops and suffering from intense competition.

## Results and Discussion <a name="results"></a>

The results from the k-means clustering show that we can categorize the neighbourhoods into 3 clusters based on the frequency of occurrence for “Coffee Shops”:

Cluster 0: Neighbourhoods with moderate number of coffee shops

Cluster 1: Neighbourhoods with low number to no existence of coffee shops

Cluster 2: Neighbourhoods with high concentration of coffee shops

The results of the clustering are visualized in the map below with cluster 0 in red colour, cluster 1 in
purple colour, and cluster 2 in mint green colour.

In [35]:
map_clusters

## Conclusion <a name="conclusion"></a>

In this project, we have gone through the process of identifying the business problem, specifying the data required, extracting and preparing the data, performing machine learning by clustering the data into 3 clusters based on their similarities, and lastly providing recommendations to the relevant stakeholders i.e. property developers and investors regarding the best locations to open a new coffee shop. To answer the business question that was raised in the introduction section, the answer proposed by this project is: The neighbourhoods in cluster 1 are the most preferred locations to open a new coffee shop. The findings of this project will help the relevant stakeholders to capitalize on the opportunities on high potential locations while avoiding overcrowded areas in their decisions to open a new coffee shop.